# Decision Tree from Scratch

In [26]:
from random import seed
from math import log
from statistics import mode
from random import randrange
from csv import reader

## Create dataset

In [27]:
dataset = [[2.771244718, 1.784783929, 0],
           [1.728571309, 1.169761413, 0],
           [3.678319846, 2.81281357, 0],
           [3.961043357, 2.61995032, 0],
           [2.999208922, 2.209014212, 0],
           [7.497545867, 3.162953546, 1],
           [9.00220326, 3.339047188, 1],
           [7.444542326, 0.476683375, 1],
           [10.12493903, 3.234550982, 1],
           [6.642287351, 3.319983761, 1]]


## Choose best splits using heuristic Funtion: Gini index and Cross-Entropy

In [28]:
def gini_index(groups, classes):
    """
    Gini_index(S,A) = Sum(|Si|/|S| * (1 - Sum(Pj*Pj))),
    where,
            A: Attribute A
            S: Root group
            Si: Child ith when apply attribute A
            Pj: % of class j
    Complexity: O(#groups x #classes)

    Args:
            `groups`: A list of groups after choose Attribute A
            `classes`: A list of classes' values
    Return:
            Gini_index(S,A)
    For example,
    >>> gini_index([[[1,0],[2,0]],[[0,1],[1,1],[2,1]]], [0,1])
    >>> 0.0
    """
    # count all samples at split point
    n_instances = float(sum([len(group) for group in groups]))
    # sum weighted Gini index for each group
    gini = 0.0
    for group in groups:
        size = float(len(group))
        # avoid divide by zero
        if size == 0:
            continue
        score = 0.0
        # score the group based on the score for each class
        for class_val in classes:
            p = [row[-1] for row in group].count(class_val) / size
            score += p * p
        # weight the group score by its relative size
        gini += (1.0 - score) * (size / n_instances)
    return gini

# Another way: calculate the entropy (slow)
def entropy(groups, classes):
    """
    Entropy(S,A) = Sum(|Si|/|S| * E(Si))
    E(Si) = sum (-Pj log(Pj))
    """
    # count all samples at split point
    n_instances = float(sum([len(group) for group in groups]))
    # sum weighted Gini index for each group
    entropy = 0.0
    for group in groups:
        size = float(len(group))
        # avoid divide by zero
        if size == 0:
            continue
        score = 0.0
        # score the group based on the score for each class
        for class_val in classes:
            p = [row[-1] for row in group].count(class_val) / size
            if p == 0 or p == 1:
                score = 0
            else:
                score -= p * log(p)
        # weight the group score by its relative size
        entropy += score * (size / n_instances)
    return entropy

In [29]:
entropy([[[1,0],[2,0]],[[0,0],[1,1],[2,1]]], [0,1]), \
gini_index([[[1,0],[2,0]],[[0,0],[1,1],[2,1]]], [0,1])

(0.38190850097688767, 0.26666666666666666)

In [30]:
def test_split(index, value, dataset):
    """
    Split a dataset based on an attribute and an attribute value.
    Complexity: O(#rows)
    """
    left, right = list(), list()
    for row in dataset:
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    return left, right

In [31]:
def get_split(dataset):
    """
    This heuristic algorithm loops all rows (samples) and column (attributes), thus it is painful for computing.
    Complexity: O(#rows^2 x #columns) x O(gini_index)
    """
    classes = list(set(row[-1] for row in dataset))
    attributes = len(dataset[0]) - 1
    retAttribute, retValue, retGroups, min_gini = None, None, None, 999
    for row in dataset:
        for attribute in range(attributes):
            groups = test_split(attribute, row[attribute], dataset)
            gini = gini_index(groups, classes)
            if gini < min_gini:
                min_gini = gini
                retAttribute, retValue, retGroups = attribute, row[attribute], groups
    return {'attribute': retAttribute, 'value': retValue, 'groups': retGroups}

get_split(dataset)


{'attribute': 0,
 'value': 6.642287351,
 'groups': ([[2.771244718, 1.784783929, 0],
   [1.728571309, 1.169761413, 0],
   [3.678319846, 2.81281357, 0],
   [3.961043357, 2.61995032, 0],
   [2.999208922, 2.209014212, 0]],
  [[7.497545867, 3.162953546, 1],
   [9.00220326, 3.339047188, 1],
   [7.444542326, 0.476683375, 1],
   [10.12493903, 3.234550982, 1],
   [6.642287351, 3.319983761, 1]])}

In [32]:
def to_terminal(group):
    assert group, 'group should not be empty'
    outcomes = [row[-1] for row in group]
    return mode(outcomes)  # most frequency
    
group = [[1, 2, 0], [1, 2, 1], [1, 2, 1]]
to_terminal(group)


1

In [33]:
def split(node, max_depth, min_size, depth):
    left, right = node['groups']
    del(node['groups'])
    # check for a no split
    if not left or not right:  # left or right is empty list
        node['left'] = node['right'] = to_terminal(left + right)
        return
    # check for max depth
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    # process left child
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left)
        split(node['left'], max_depth, min_size, depth+1)
    # process right child
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right)
        split(node['right'], max_depth, min_size, depth+1)


## Build a decision tree

In [34]:
# Build a decision tree
def build_tree(train_set, max_depth, min_size):
    root = get_split(train_set)
    split(root, max_depth, min_size, 0)
    return root

# Print a decision tree
def print_tree(node, depth=0):
    if isinstance(node, dict):
        print('%s[X%d < %.3f]' %
              ((depth*'-', (node['attribute']+1), node['value'])))
        print_tree(node['left'], depth+2)
        print_tree(node['right'], depth+2)
    else:
        print('%s[%s]' % ((depth*'-', node)))  # This is cool!


tree = build_tree(dataset, 4, 1)
print_tree(tree)
# print(tree)

[X1 < 6.642]
--[X1 < 2.771]
----[0]
----[X1 < 2.771]
------[0]
------[0]
--[X1 < 7.498]
----[X1 < 7.445]
------[1]
------[1]
----[X1 < 7.498]
------[1]
------[1]


## Prediction

In [35]:
def predict(tree, input_row):  # input is row of testset
    if input_row[tree['attribute']] < tree['value']:
        if isinstance(tree['left'], dict):
            return predict(tree['left'], input_row)
        else:
            return tree['left']
    else:
        if isinstance(tree['right'], dict):
            return predict(tree['right'], input_row)
        else:
            return tree['right']


for row in dataset:
    ret = predict(tree, row)
    print('Expected: %d, Predicted: %d' % (row[-1], ret))


Expected: 0, Predicted: 0
Expected: 0, Predicted: 0
Expected: 0, Predicted: 0
Expected: 0, Predicted: 0
Expected: 0, Predicted: 0
Expected: 1, Predicted: 1
Expected: 1, Predicted: 1
Expected: 1, Predicted: 1
Expected: 1, Predicted: 1
Expected: 1, Predicted: 1


## Work with Real Data

In [36]:
# Load a CSV file
def load_csv(filename):
    file = open(filename, "rt")
    lines = reader(file)
    dataset = list(lines)
    return dataset

# Convert string column to float
def str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column])

# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for i in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])  # concatenate lists of lists to a list
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            # test_set use to predict => no need to hold [class] data
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
    return scores


def decision_tree(train_set, test_set, max_depth, min_size):
    root = build_tree(train_set, max_depth, min_size)
    prediction = list()
    for row in test_set:
        prediction.append(predict(root, row))
    return prediction

In [37]:
# Test CART on Bank Note dataset
seed(1)
# load and prepare data
filename = 'data/BankNote_Authentication.csv'
dataset = load_csv(filename)
# remove the string attributes
dataset.pop(0) 

# convert string attributes to integers
for i in range(len(dataset[0])):
    str_column_to_float(dataset, i)
# evaluate algorithm
n_folds = 5
max_depth = 5  # maximum depth of tree
min_size = 10  # minimum size of 1 group
scores = evaluate_algorithm(
    dataset, decision_tree, n_folds, max_depth, min_size)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

Scores: [98.17518248175182, 97.44525547445255, 98.90510948905109, 98.54014598540147, 97.8102189781022]
Mean Accuracy: 98.175%


## Conclusion
Very high accuracy

## Refs
https://machinelearningmastery.com/implement-decision-tree-algorithm-scratch-python/